In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.metrics import mean_squared_error as mse
from sklearn.svm import SVR
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\AURIMASSilva\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df = pd.read_csv('ready_returns.csv', header = None)
df.columns = ['date', 'returns']
df.returns = df.returns
data, data2 = df['returns'].values, df['returns'].values
assert len(data) == len(df) == len(data2)
cut_off = df[df['date'] == '2018-03-06']['returns'].values[0]

In [3]:
data

array([ 2.30633546,  4.01737101,  2.03516993, ..., -8.07753641,
       -0.6181936 , -2.57139795])

In [4]:
length_param = 3
data_gen = TimeseriesGenerator(np.append(data,0.00), np.append(data2,0.00),
                               length=length_param, sampling_rate=1,
                               batch_size=1)

In [5]:
features = []
targets = []
i = 0
while True:
    try:
        features.append(list(data_gen[i][0][0]))
        targets.append(data_gen[i][1][0])
        i += 1
    except:
        break

        
features = np.array(features)
targets = np.array(targets)

In [6]:
features[-1]
print(targets[-1])
print(data[-1])


-2.5713979508094518
-2.5713979508094518


In [ ]:
#targets[np.argwhere(targets == cut_off)[0][0]]

In [ ]:
cut_it = np.argwhere(targets == cut_off)[0][0]
print(cut_it)
x_test, y_test = features[cut_it:], targets[cut_it:]
x_hold, y_hold = features[:cut_it], targets[:cut_it]

In [ ]:
x_train, x_valid = x_hold[:int(len(x_hold)*.9)], x_hold[int(len(x_hold)*.9):]
y_train, y_valid = y_hold[:int(len(y_hold)*.9)], y_hold[int(len(y_hold)*.9):]

In [ ]:
assert len(x_train) + len(x_valid) == len(x_hold)

---

In [ ]:
reg = SVR(kernel = 'poly')

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_valid)

In [ ]:
np.sqrt(mse(y_valid,y_hat))

In [ ]:
reg = SVR()

from sklearn.model_selection import GridSearchCV
parameters = [{'C' : [0.01,0.1, 0.1,10,100,1000], 'kernel' : ['linear'], 'epsilon' : [0.005, 0.01,0.05,0.1,0.2,0.5]}]
grid_search = GridSearchCV(estimator = reg,
                           param_grid = parameters,
                           scoring = 'neg_mean_squared_error',
                           cv = None,
                           n_jobs = -1)

grid_search = grid_search.fit(x_train, y_train)

bestaccuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

In [ ]:
print(bestaccuracy)

In [ ]:
best_parameters

In [ ]:
reg = SVR(kernel= 'rbf', epsilon = 0.1, C = 0.01)

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
y_hat = reg.predict(x_test)

In [ ]:
np.sqrt(mse(y_test,y_hat))

In [ ]:
np.sqrt(mse(y_valid, y_hat))

In [ ]:
y_hat = reg.predict(x_test)

In [ ]:
np.sqrt(mse(y_test, y_hat))

In [ ]:
y_test.var()